In [1]:
import os

In [2]:
%pwd

'd:\\Text-summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text-summarizer'

In [40]:
%pip install --upgrade fsspec
%pip install --upgrade datasets

Note: you may need to restart the kernel to use updated packages.
  Using cached pyarrow-21.0.0-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached pyarrow-21.0.0-cp311-cp311-win_amd64.whl (26.2 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

  Attempting uninstall: tqdm

    Found existing installation: tqdm 4.65.0

    Uninstalling tqdm-4.65.0:

   ---------------------------------------- 0/3 [tqdm]
   ---------------------------------------- 0/3 [tqdm]
      Successfully uninstalled tqdm-4.65.0
   ---------------------------------------- 0/3 [tqdm]
   ---------------------------------------- 0/3 [tqdm]
   ---------------------------------------- 0/3 [tqdm]
   ---------------------------------------- 0/3 [tqdm]
   ---------------------------------------- 0/3 [tqdm]
   ---------------------------------------- 0/3 [tqdm]
   ---------------------------------------- 0/3 [tqdm]
   ------------------------------------

  You can safely remove it manually.


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

d:\Text-summarizer\texts\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
print(os.path.exists("artifacts/data_ingestion/samsum_dataset"))
print(os.path.exists("artifacts/data_ingestion"))
for root,dirs,files in os.walk(".",topdown=True):
    if"samsum_dataset" in dirs:
        print(os.path.join(root, "samsum_dataset"))

True
True
.\artifacts\data_ingestion\samsum_dataset


In [10]:
dataset = load_from_disk("artifacts/data_ingestion/samsum_dataset/test")
print(dataset)

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})


In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        abs_path = os.path.abspath(self.config.data_path)
        print(f"The root directory is :{self.config.root_dir}")
        print(f"The data path is:{self.config.data_path}")
        print(f"The absolute path is:{abs_path}")
        dataset_samsum = load_from_disk(abs_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        print(self.config.root_dir)

In [12]:
path = os.path.abspath("artifacts/data_ingestion/samsum_dataset")
print(os.listdir(path))


['dataset_dict.json', 'test', 'train', 'validation']


In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-09-21 09:44:01,301: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-21 09:44:01,308: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-21 09:44:01,308: INFO: common: created directory at: artifacts]
[2025-09-21 09:44:01,308: INFO: common: created directory at: artifacts/data_transformation]


d:\Text-summarizer\texts\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The root directory is :artifacts/data_transformation
The data path is:artifacts/data_ingestion/samsum_dataset
The absolute path is:d:\Text-summarizer\artifacts\data_ingestion\samsum_dataset


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\Text-summarizer\texts\Lib\site-packages\transformers\tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 48703.82 examples/s]

artifacts/data_transformation
